[구글 코랩(Colab)에서 실행하기](https://colab.research.google.com/github/lovedlim/tensorflow/blob/main/Part%204/4.8_Gan_Mnist.ipynb)

## Import

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pylab as plt
import cv2
from tqdm.notebook import tqdm

## 파라미터 설정

In [ ]:
img_shape = (28, 28, 1)
z_dim = 100
row_num = 8
col_num = 8
batch_size = row_num * col_num
epoch_num = 10
learning_rate = 0.0001
class_num = 10

## MNIST 데이터셋

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

11493376/11490434 [==============================] - 0s 0us/step


## Discriminator 모델

In [ ]:
i=tf.keras.Input(shape=img_shape)
out = tf.keras.layers.Conv2D(16, 3, 2, padding='same')(i)
out = tf.keras.layers.Conv2D(32, 3, 2, padding='same')(out)
out = tf.keras.layers.Conv2D(64, 3, 2, padding='same')(out)
out = tf.keras.layers.Flatten()(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.Dense(1024, activation='tanh')(out)
out = tf.keras.layers.Dense(1, activation='sigmoid')(out)
d_model = tf.keras.Model(inputs=[i],  outputs=[out])

d_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 14, 14, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          18496     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dense (Dense)                (None, 1024)              104960

## Generator 모델

In [ ]:
i=tf.keras.Input(shape=(z_dim, ))
out = tf.keras.layers.Dense(1024, activation='tanh')(i)
out = tf.keras.layers.Dense(7*7*32, activation='tanh')(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.Reshape((7, 7, 32))(out)
out = tf.keras.layers.Conv2DTranspose(16, 3, 2, padding='same')(out)
out = tf.keras.layers.Conv2DTranspose(1, 3, 2, padding='same')(out)
out = tf.keras.layers.Activation('sigmoid')(out)
g_model = tf.keras.Model(inputs=[i],  outputs=[out])

g_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              103424    
_________________________________________________________________
dense_3 (Dense)              (None, 1568)              1607200   
_________________________________________________________________
batch_normalization_1 (Batch (None, 1568)              6272      
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 16)        4624      
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 28, 28, 1)         145 

## GAN 학습

In [ ]:
# 옵티마이저
opt = tf.keras.optimizers.Adam(learning_rate)

# Discriminator 손실함수
d_loss = tf.keras.losses.BinaryCrossentropy()

In [ ]:
fcc=cv2.VideoWriter_fourcc(*'DIVX')
out=cv2.VideoWriter('hjk_gan_mnist.avi', fcc, 1.0, (28*row_num, 28 * col_num))

# 1 Epoch에 batch size를 고려하여 학습할 수를 구한다.
batch_count =x_train.shape[0]//batch_size

for e in range(epoch_num):
    for _ in tqdm(range(batch_count)):
        
        # z는 Noise 또는 Latent Vector라 불리우는 값이다.
        # 위조지폐(Fake Image)를 만드는 재료라고 생각하자. 
        z = np.random.uniform(-1.0, 1.0, (batch_size, z_dim))
        # 재료(z)를 가지고 가짜 이미지를 만든다.
        # 그리고 가짜(0)라고 라벨을 만들자
        f_img = g_model.predict(z)
        f_label = np.zeros((batch_size, 1))
        
        # Gradient Tape를 그리고,  Discriminator Loss(binary cross entropy)를
        # 이용하여 위조지폐(Fake Image)는 가짜(0)라고 Discriminator를 학습하자
        with tf.GradientTape() as tape:
            pred = d_model(f_img)
            loss = d_loss(f_label, pred)
        vars = d_model.trainable_variables
        grad = tape.gradient(loss, vars)
        opt.apply_gradients(zip(grad, vars))
        
        # x_train에서 랜덤하게 batch size만큼 데이터를 가지고 온다.
        # 그리고 진짜(1)이라고 라벨을 만든다.
        batch_num=np.random.randint(0,  x_train.shape[0],  size=batch_size)
        r_img = x_train[batch_num]
        r_label = np.ones((batch_size, 1))
        
        # Gradient Tape를 그리고,  Discriminator Loss(binary cross entropy)를
        # 이용하여 진짜 지폐(Real Image)는 진짜(1)라고 Discriminator를 학습하자

        with tf.GradientTape() as tape:
            pred = d_model(r_img)
            loss = d_loss(r_label, pred)
        vars = d_model.trainable_variables
        grad = tape.gradient(loss, vars)
        opt.apply_gradients(zip(grad, vars))

        # 이번엔 위조지폐범(Generator)를 학습해보자
        # Gradient Tape를 그리고,  Discriminator Loss(binary cross entropy)를
        # 이용하여 가짜 지폐(Real Image)는 진짜(1)라고 Discriminator에게 속인다.
        # 그리고 Discriminator가 틀리다고 생각되는 부분을 
        # 위조지폐범에게 정보를 전달하여 그부분을 수정하도록 한다.
        with tf.GradientTape() as tape:
            f_img = g_model(z)
            pred = d_model(f_img)
            loss = d_loss(r_label, pred)
        vars = g_model.trainable_variables
        grad = tape.gradient(loss, vars)
        opt.apply_gradients(zip(grad, vars))     
        
        # 학습 시 마다 중간 결과를 이미지로 만들고,  이미지를 붙여 동영상으로 저정하자.
        sample_img = np.zeros((28*row_num, 28 * col_num))
        f_img = np.resize(f_img, (row_num, col_num, 28, 28))
        for i in range(row_num):
            for j in range(col_num):
                sample_img[i * 28:i * 28 +28, j * 28:j * 28 +28] = f_img[i, j, :, :]
        sample_img = np.uint8(sample_img * 255.)
        sample_img = cv2.applyColorMap(sample_img, cv2.COLORMAP_HOT)
        out.write(sample_img)
    
    print(e, "완료")
out.release()


0 완료



1 완료



2 완료



3 완료



4 완료



5 완료



6 완료



7 완료



8 완료



9 완료


## Conditional GAN

In [ ]:
# Label을 입력값으로 받을 수 있도록 Discriminator 모델 수정
i = tf.keras.Input(shape=img_shape)
l_i = tf.keras.Input(shape=(1, ), dtype=tf.int32)
l_out = tf.one_hot(l_i, class_num)
l_out = tf.keras.layers.Dense(28*28*1)(l_out)
l_out = tf.keras.layers.Reshape((28, 28, 1))(l_out) 
out = tf.keras.layers.Add()([i, l_out])
out = tf.keras.layers.Conv2D(16, 3, 2, padding='same')(out)
out = tf.keras.layers.Conv2D(32, 3, 2, padding='same')(out)
out = tf.keras.layers.Conv2D(64, 3, 2, padding='same')(out)
out = tf.keras.layers.Flatten()(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.Dense(1024, activation='tanh')(out)
out = tf.keras.layers.Dense(1, activation='sigmoid')(out)
d_model = tf.keras.Model(inputs=[i, l_i],  outputs=[out])

# Label을 입력값으로 받을 수 있도록 Generator모델 수정
i=tf.keras.Input(shape=(z_dim, ))
l_i = tf.keras.Input(shape=(1, ), dtype=tf.int32)
l_out = tf.one_hot(l_i, class_num)
l_out = tf.keras.layers.Dense(z_dim)(l_out)
l_out = tf.keras.layers.Reshape((z_dim, ))(l_out) 
out = tf.keras.layers.Add()([i, l_out])
out = tf.keras.layers.Dense(1024, activation='tanh')(out)
out = tf.keras.layers.Dense(7*7*32, activation='tanh')(out)
out = tf.keras.layers.BatchNormalization()(out)
out = tf.keras.layers.Reshape((7, 7, 32))(out)
out = tf.keras.layers.Conv2DTranspose(16, 3, 2, padding='same')(out)
out = tf.keras.layers.Conv2DTranspose(1, 3, 2, padding='same')(out)
out = tf.keras.layers.Activation('sigmoid')(out)
g_model = tf.keras.Model(inputs=[i, l_i],  outputs=[out])

# Data를 생성하고 판단할 때 Label 값을 받도록 수정 
fcc=cv2.VideoWriter_fourcc(*'DIVX')
out=cv2.VideoWriter('cgan_mnist.avi', fcc, 10.0, (28*row_num, 28 * col_num))

batch_count =x_train.shape[0]//batch_size

for e in range(epoch_num):
    for _ in tqdm(range(batch_count)):
        
        # f_y값은 0~9까지 임의값을 원핫 인코딩한 값이다.
        # f_y값도 Ganerator의 인풋값으로 추가하자
        z = np.random.uniform(-1.0, 1.0, (batch_size, z_dim))
        f_y = np.random.randint(0, class_num, size=batch_size)
        f_y = np.reshape(f_y, (batch_size, 1))
        f_img = g_model.predict([z, f_y])
        f_label = np.zeros((batch_size, 1))
        
        # 위조지폐(Fake Image)와 라벨 모두를 인풋으로 받아서
        # 가짜라고 학습한다.
        # 예를 들어 5000원짜리 위조지폐라고 학습한다. 
        with tf.GradientTape() as tape:
            pred = d_model([f_img, f_y])
            loss = d_loss(f_label, pred)
        vars = d_model.trainable_variables
        grad = tape.gradient(loss, vars)
        opt.apply_gradients(zip(grad, vars))
        
        # 실제지폐(Real Image)와 라벨 모두를 인풋으로 받아서
        # 진짜라고 학습한다.
        # 예를 들어 5000원짜리 진짜 지폐라고 학습한다. 
        batch_num=np.random.randint(0, x_train.shape[0], size=batch_size)
        r_img = x_train[batch_num]
        r_y = y_train[batch_num]
        r_label = np.ones((batch_size, 1))

        with tf.GradientTape() as tape:
            pred = d_model([r_img, r_y])
            loss = d_loss(r_label, pred)
        vars = d_model.trainable_variables
        grad = tape.gradient(loss, vars)
        opt.apply_gradients(zip(grad, vars))

        f_y=[i%class_num for i in range(batch_size)]
        f_y = np.reshape(f_y, (batch_size, 1))

        # 이제 가짜 지폐와 가짜 라벨을 경찰에게 보여주고
        # 이 지폐가 진짜라고 학습하고,  그 차이점을 위조지폐범에게 알려준다.
        # 그리고 위조지폐범은 그 차이를 수정해 나간다.

        with tf.GradientTape() as tape:
            f_img = g_model([z, f_y])
            pred = d_model([f_img, f_y])
            loss = d_loss(r_label, pred)
        vars = g_model.trainable_variables
        grad = tape.gradient(loss,  vars)
        opt.apply_gradients(zip(grad,  vars))     
        
        sample_img = np.zeros((28*row_num, 28 * col_num))
        f_img = np.resize(f_img, (row_num, col_num, 28, 28))
        for i in range(row_num):
            for j in range(col_num):
                sample_img[i * 28:i * 28 +28, j * 28:j * 28 +28] = f_img[i, j, :, :]
        sample_img = np.uint8(sample_img * 255.)
        sample_img = cv2.applyColorMap(sample_img, cv2.COLORMAP_HOT)
        out.write(sample_img)
    
    print(e, "완료")
out.release()


0 완료



1 완료



2 완료



3 완료



4 완료



5 완료



6 완료



7 완료



8 완료



9 완료
